# Rook und Ceph

**Ceph** ist ein verteiltes, fehlertolerantes und hochskalierbares Speichersystem, das Objektspeicher, Blockspeicher (RBD) und verteilte Dateisysteme (CephFS) bereitstellt. Es nutzt das **CRUSH-Algorithmus**, um Daten effizient und ohne zentralen Flaschenhals auf Cluster-Knoten zu verteilen. Ceph wird häufig in Cloud- und Virtualisierungsumgebungen wie OpenStack oder Kubernetes eingesetzt.  

**Rook** ist ein **Cloud-Native Storage Orchestrator**, der Ceph in Kubernetes-Clustern automatisiert bereitstellt, verwaltet und überwacht. Rook vereinfacht die Bereitstellung und Verwaltung von Ceph, indem es als Operator in Kubernetes fungiert und automatisch Storage-Pools, Monitore und OSDs (Object Storage Daemons) verwaltet.  

**Vorteile von Rook und Ceph:**  
- **Skalierbarer, verteilter Speicher** für Block-, Datei- und Objektdaten  
- **Automatisches Self-Healing** und Fehlertoleranz  
- **Nahtlose Kubernetes-Integration** für Stateful-Workloads  
- **Open-Source & Software-Defined Storage**, unabhängig von proprietärer Hardware  

### rbd Modul

Das **rbd-Modul** des Linux-Kernels ist ein Treiber für das **RADOS Block Device (RBD)**, das in **Ceph**-Clustern verwendet wird. Es ermöglicht den Zugriff auf verteilte, skalierbare Speichergeräte, die über das Ceph-Cluster-Storage-System bereitgestellt werden. Das rbd-Modul stellt diese Speicher als reguläre Blockgeräte im System zur Verfügung, sodass sie wie lokale Festplatten verwendet werden können.  

Zu den Hauptfunktionen gehören:  
- Direkte Anbindung an Ceph-Cluster über das Netzwerk  
- Unterstützung für Snapshots und Klone  
- Dynamische Skalierung und hohe Verfügbarkeit  
- Effiziente Lastverteilung durch verteilten Speicher  

Das rbd-Modul wird typischerweise in Cloud-Umgebungen und Virtualisierungslösungen wie **OpenStack** oder **Kubernetes** genutzt, um hochverfügbaren und redundanten Speicher bereitzustellen.

---

Bevor wir rook-ceph enablen, muss rdb im Linux Kernel aktiviert sein:

In [1]:
%%bash
sudo apt-get install ceph-common -y
lsmod | grep ceph
sudo modprobe ceph

sudo modprobe rbd
lsmod | grep rbd

rbd                   126976  0
libceph               544768  2 ceph,rbd


In [10]:
%%bash
git clone --single-branch --branch v1.16.3 https://github.com/rook/rook.git
cd rook/deploy/examples
kubectl create -f crds.yaml -f common.yaml -f operator.yaml

Cloning into 'rook'...
Note: switching to 'fd9e649070e2ed0500eb2e5d8690664eae7cf90b'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



customresourcedefinition.apiextensions.k8s.io/cephblockpoolradosnamespaces.ceph.rook.io created
customresourcedefinition.apiextensions.k8s.io/cephblockpools.ceph.rook.io created
customresourcedefinition.apiextensions.k8s.io/cephbucketnotifications.ceph.rook.io created
customresourcedefinition.apiextensions.k8s.io/cephbuckettopics.ceph.rook.io created
customresourcedefinition.apiextensions.k8s.io/cephclients.ceph.rook.io created
customresourcedefinition.apiextensions.k8s.io/cephclusters.ceph.rook.io created
customresourcedefinition.apiextensions.k8s.io/cephcosidrivers.ceph.rook.io created
customresourcedefinition.apiextensions.k8s.io/cephfilesystemmirrors.ceph.rook.io created
customresourcedefinition.apiextensions.k8s.io/cephfilesystems.ceph.rook.io created
customresourcedefinition.apiextensions.k8s.io/cephfilesystemsubvolumegroups.ceph.rook.io created
customresourcedefinition.apiextensions.k8s.io/cephnfses.ceph.rook.io created
customresourcedefinition.apiextensions.k8s.io/cephobjectrea

Dann können wir rook-ceph enablen

In [ ]:
%%bash
#sudo microk8s enable rook-ceph

In [ ]:
! kubectl get pods -n rook-ceph

### Host Storage Cluster

Um rook-ceph auszuprobieren, verwenden wir die Variante "Host Storage Cluster".

In einem "Host Storage Cluster" konfiguriert Rook Ceph so, dass Daten direkt auf dem Host gespeichert werden. 

Der `allowMultiplePerNode: true` sollte `false` sein, damit die Daten über die drei Hosts verteilt werden.

In [ ]:
%%bash
kubectl delete -f - <<EOF
#################################################################################################################
# Define the settings for the rook-ceph cluster with common settings for a small test cluster.
# All nodes with available raw devices will be used for the Ceph cluster. One node is sufficient
# in this example.

# For example, to create the cluster:
#   kubectl create -f crds.yaml -f common.yaml -f operator.yaml
#   kubectl create -f cluster-test.yaml
#################################################################################################################
apiVersion: ceph.rook.io/v1
kind: CephCluster
metadata:
  name: my-cluster
  namespace: rook-ceph # namespace:cluster
spec:
  dataDirHostPath: /var/lib/rook
  cephVersion:
    image: quay.io/ceph/ceph:v19
    allowUnsupported: true
  mon:
    count: 1
    allowMultiplePerNode: true
  mgr:
    count: 1
    allowMultiplePerNode: true
    modules:
      - name: rook
        enabled: true
  dashboard:
    enabled: true
  crashCollector:
    disable: true
  storage:
    useAllNodes: true
    useAllDevices: true
    allowDeviceClassUpdate: true
    allowOsdCrushWeightUpdate: false
    #deviceFilter:
    #config:
    #  deviceClass: testclass
  monitoring:
    enabled: false
  healthCheck:
    daemonHealth:
      mon:
        interval: 45s
        timeout: 600s
  priorityClassNames:
    all: system-node-critical
    mgr: system-cluster-critical
  disruptionManagement:
    managePodBudgets: true
  cephConfig:
    global:
      osd_pool_default_size: "1"
      mon_warn_on_pool_no_redundancy: "false"
      bdev_flock_retry: "20"
      bluefs_buffered_io: "false"
      mon_data_avail_warn: "10"
---
apiVersion: ceph.rook.io/v1
kind: CephBlockPool
metadata:
  name: builtin-mgr
  namespace: rook-ceph # namespace:cluster
spec:
  name: .mgr
  replicated:
    size: 1
    requireSafeReplicaSize: false
EOF

cephcluster.ceph.rook.io "my-cluster" deleted
cephblockpool.ceph.rook.io "builtin-mgr" deleted


In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: ceph.rook.io/v1
kind: CephCluster
metadata:
  name: rook-ceph
  namespace: rook-ceph
spec:
  cephVersion:
    # see the "Cluster Settings" section below for more details on which image of ceph to run
    image: quay.io/ceph/ceph:v17.2.6
  dataDirHostPath: /var/lib/rook
  mon:
    count: 3
    allowMultiplePerNode: true
  storage:
    useAllNodes: true
    useAllDevices: true
EOF


Dieser Eintrag triggert den Rook Operator welcher die weiteren Storage Driver aktiviert

In [ ]:
%%bash
kubectl get pods --namespace rook-ceph


### Links

* [Host Storage Cluster](https://rook.io/docs/rook/v1.11/CRDs/Cluster/host-cluster/)